In [1]:
# libraries
import pandas as pd
import numpy as np
import folium
from folium import plugins as fol_pi
import ipywidgets as widgets
from ipywidgets import interact
import IPython.display as IPdisp


In [2]:
# base AMR metric
def Transmission_score_formula(x, y, z):
    return 2*(x+y+z)

def Access_score_formula(x, y):
    if x < 3:
        return 2*y
    elif x < 5:
        return 1 + 2*y
    else:
        return 2 + 2*y

def Herd_size_score(herd):
    if herd < 1:
        return 0
    elif herd <10:
        return 1
    elif herd < 50:
        return 2
    elif herd < 100:
        return 3
    else:
        return 4

def Disease_score_formula(c_herd, b_herd, g_herd, c_dis, b_dis, g_dis):
    return (Herd_size_score(c_herd)+Herd_size_score(b_herd)+Herd_size_score(g_herd)+c_dis+b_dis+g_dis)/3

def Use_score_formula(u, v, w, x, y, z):
    return (u + v + w + x + y + z)

def Overall_score_formula(tra, acc, dis, use):
    return (tra + acc + dis + use)

def base_AMR_metr(manip_amr_data):
    manip_amr_data['Transmission score'] = manip_amr_data.apply(lambda row: Transmission_score_formula(row['Transmission_A'], row['Transmission_B'], row['Transmission_C']), axis=1)        
    manip_amr_data['Access score'] = manip_amr_data.apply(lambda row: Access_score_formula(row['Frequency of antibiotic purchase'], row['Use of trained animal healthcare professional']), axis=1)        
    manip_amr_data['Disease score'] = manip_amr_data.apply(lambda row: Disease_score_formula(row['Livestock_1 herd size'], row['Livestock_2 herd size'], row['Livestock_3 herd size'], row['Livestock_1 Disease priority'], row['Livestock_2 Disease priority'], row['Livestock_3 Disease priority']), axis=1)        
    manip_amr_data['Use score'] = manip_amr_data.apply(lambda row: Use_score_formula(row['Mixing antibiotics in the same animal'], row['Not completing course'], row['Not measuring dose accurately'], row['Using for prevention'], row['Drink milk from treated animals'], row['Eat meat from treated animals']), axis=1)
    # Now four component scores are combined for final AMR Metric
    manip_amr_data['Overall score'] = manip_amr_data.apply(lambda row: Overall_score_formula(row['Transmission score'], row['Access score'], row['Disease score'], row['Use score']), axis=1)
    return manip_amr_data


In [3]:
# User manipulation AMR Metric
def user_Transmission_score_formula(dung_field, shed_clean, health_visit, add_dung, add_clean, add_visit):
    return max(0,min(2*(dung_field+add_dung),2)) + max(0,min(2*(shed_clean+add_clean),2)) + max(0,min(2*(health_visit+add_visit),2)) #to be applied at community level

def user_Access_score_formula(freq_purch, trained_risk, add_freq, add_trained):
    risk = max(0,min(trained_risk + add_trained,2))
    freq = freq_purch + add_freq
    if freq < 3:
        return 0 + 2*risk
    elif freq < 5:
        return 1 + 2*risk
    else:
        return 2 + 2*risk

def user_Disease_score_formula(c_herd, b_herd, g_herd, c_dis, b_dis, g_dis,\
                               add_c_herd, add_b_herd, add_g_herd,\
                              add_c_dis, add_b_dis, add_g_dis):
    user_c_dis = min(2,max(0,c_dis + add_c_dis))
    user_b_dis = min(2,max(0,b_dis + add_b_dis))
    user_g_dis = min(2,max(0,g_dis + add_g_dis))    
    return (Herd_size_score(c_herd+add_c_herd)+Herd_size_score(b_herd+add_b_herd)+Herd_size_score(g_herd+add_g_herd)+\
            user_c_dis+user_b_dis+user_g_dis)/3


def user_Use_score_formula(mix_anti, not_comp, not_meas, use_prev, drink_mlk, eat_meat,\
                          add_mix_anti, add_not_comp, add_not_meas, add_use_prev, add_drink_mlk, add_eat_meat):
    return max(0,min((mix_anti+add_mix_anti),1)) + max(0,min((not_comp+add_not_comp),1)) + max(0,min((not_meas+add_not_meas),1))\
+ max(0,min((use_prev+add_use_prev),1)) + max(0,min((drink_mlk+add_drink_mlk),1)) + max(0,min((eat_meat+add_eat_meat),1))

#to be applied at community level


def user_Overall_score_formula(tra, acc, dis, use):
    return (tra + acc + dis + use)
                                                                                    
                                                                                    
# Next we apply Access and Disease score formula at household level, then take the mean to allow Transmission and Use scored to be manipulated by user

def user_AMR_comp(amr_input_data_raw, add_dung, add_clean,\
                  add_visit, add_freq, add_trained,\
                 add_c_herd, add_b_herd, add_g_herd,\
                  add_c_dis, add_b_dis, add_g_dis,\
                 add_mix_anti, add_not_comp, add_not_meas,\
                  add_use_prev, add_drink_mlk, add_eat_meat):
    
    amr_input_data = amr_input_data_raw.copy()
    
    #first apply the household level user manipulation scores
    amr_input_data['Access score'] = amr_input_data.apply(lambda row: user_Access_score_formula(row['Frequency of antibiotic purchase'], row['Use of trained animal healthcare professional'], add_freq, add_trained), axis=1)        
    amr_input_data['Disease score'] = amr_input_data.apply(lambda row: user_Disease_score_formula(row['Livestock_1 herd size'], row['Livestock_2 herd size'], row['Livestock_3 herd size'], row['Livestock_1 Disease priority'], row['Livestock_2 Disease priority'], row['Livestock_3 Disease priority'], add_c_herd, add_b_herd, add_g_herd, add_c_dis, add_b_dis, add_g_dis), axis=1)        
    #next groupby to community
    manip_amr_data_community = amr_input_data.groupby(['Community']).mean()
    
    #now apply two community level scores
    manip_amr_data_community['Transmission score'] = manip_amr_data_community.apply(lambda row: user_Transmission_score_formula(row['Transmission_A'], row['Transmission_B'], row['Transmission_C'], add_dung, add_clean, add_visit), axis=1)        
    manip_amr_data_community['Use score'] = manip_amr_data_community.apply(lambda row: user_Use_score_formula(row['Mixing antibiotics in the same animal'], row['Not completing course'], row['Not measuring dose accurately'], row['Using for prevention'], row['Drink milk from treated animals'], row['Eat meat from treated animals'], add_mix_anti, add_not_comp, add_not_meas, add_use_prev, add_drink_mlk, add_eat_meat), axis=1)                                                                                     
    
                                                                                    
    # All four components scored, we can find the overall                                                                                
    manip_amr_data_community['Overall score'] = manip_amr_data_community.apply(lambda row: user_Overall_score_formula(row['Transmission score'], row['Access score'], row['Disease score'], row['Use score']), axis=1)
    
    return manip_amr_data_community



In [4]:
# create colour rules for each rag score
def colour_rag_component(val):
    colour = 'red'
    if type(val) is not str and val<=4:
        colour = 'orange'
    if type(val) is not str and val<2:
        colour = 'green'
    return 'color: %s' % colour


# create colour rules for each env_smp rag score
def colour_rag_component_env_smp(val):
    colour = 'green'
    if type(val) is not str and val>0:
        colour = 'red'
    return 'color: %s' % colour

def colour_rag_table_env(val):
    colour = 'black'
    if val in ['Yes','resistant']:
        colour = 'red'
    if val in ['No','sensitive']:
        colour = 'green'
    return 'color: %s' % colour



def colour_rag_overall(val):
    colour = 'red'
    if type(val) is not str and val<=18:
        colour = 'orange'
    if type(val) is not str and val<8:
        colour = 'green'
    return 'color: %s' % colour

# Allow zoom for table view
def magnify():
    return [dict(selector="th", #TableHeaders
                 props=[("font-size", "11pt")]),
            dict(selector="td", #TableData
                 props=[('padding', "0em 3em"),
                        ('font-size', '10pt')]),
            dict(selector="th:hover",
                 props=[("font-size", "14pt")]),
            dict(selector="tr:hover td:hover",
                 props=[('max-width', '200px'),
                        ('font-size', '14pt')])]

def magnify2():
    return [dict(selector="th", #TableHeaders
                 props=[("font-size", "11pt")]),
            dict(selector="td", #TableData
                 props=[('padding', "0em 3em"),
                        ('font-size', '10pt')])]

# Adapt for map

def colour_rag_overall_marker(val):
    colour = 'red'
    if type(val) is not str and val<=18:
        colour = 'orange'
    if type(val) is not str and val<8:
        colour = 'green'
    return colour

# Formatting functions for table display

def displ_YN(x):
    if x == 1:
        return "Yes"
    else:
        return "No"
    
def displ_risk(x):
    if x == 2:
        return "High risk"
    elif x == 1:
        return "Medium risk"
    else:
        return "Low risk"
    
def displ_risk_comm(x):
    if x == 0:
        return "Low risk"
    elif x < 1:
        return "Low-Medium risk"
    elif x == 1:
        return "Medium risk"
    elif x < 2:
        return "Medium-High risk"
    else:
        return "High risk"

In [5]:
# Function for displaying Community markers with AMR Metric scores on map

def create_map(user_AMR_data, lat, lon, tiles="OpenStreetMap", zoom=13):
    m = folium.Map(location=(lat, lon), tiles=tiles, zoom_start=zoom)    

# amr_metric_community = manip_amr_data_community[['Latitude','Longitude','Transmission score','Access score','Disease score','Use score','Overall score']]
    
    for index, row in user_AMR_data.iterrows():
        #print(index, row['village'], row['Latitude'], row['Longitude'])
        html="""
        <h1 style="font-family:arial;text-align:center;">Community: {com}</h1>
        <p style="{col_ov};font-family:arial;font-size:22px;"><b>Overall score: {ov:.2f}</b></p>
        <p style="{col_tr};font-family:arial;">Transmission score: {tr:.2f}</p>
        <p style="{col_ac};font-family:arial;">Access score: {ac:.2f}</p>
        <p style="{col_di};font-family:arial;">Disease score: {di:.2f}</p>
        <p style="{col_us};font-family:arial;">Use score: {us:.2f}</p>
        """.format(com=index,ov=row['Overall score'],col_ov=colour_rag_overall(row['Overall score']),\
                   tr=row['Transmission score'],col_tr=colour_rag_component(row['Transmission score']),\
                   ac=row['Access score'],col_ac=colour_rag_component(row['Access score']),\
                   di=row['Disease score'],col_di=colour_rag_component(row['Disease score']),\
                   us=row['Use score'],col_us=colour_rag_component(row['Use score']))

        iframe = folium.IFrame(html=html,width=400, height=250)#, width=300, height=300)
        popup = folium.Popup(iframe, parse_html=True, max_width=2650)

        folium.Marker([row['Latitude'],row['Longitude']],popup=popup, tooltip=index, icon = folium.Icon(color=colour_rag_overall_marker(row['Overall score']),icon='users',prefix='fa',icon_color='white')).add_to(m)
        # Note - icon colors restricted - see help(folium.Icon), icon_colour accepts HTML - see https://www.rapidtables.com/web/color/html-color-codes.html
        # Note - for icons see https://fontawesome.com/icons?d=gallery&m=free (not all work)
    

    m.add_child(folium.LatLngPopup())#, folium.LayerControl().add_to(m)
    
    fol_pi.Fullscreen(
    position='topright',
    title='Full screen',
    title_cancel='Exit full screen',
    force_separate_button=True).add_to(m)
    
    return m



# Function for displaying Community markers with AMR Environment Samples on map

def create_map_env_samp(user_AMR_env_samp_data, lat, lon, tiles="OpenStreetMap", zoom=13):
    m = folium.Map(location=(lat, lon), tiles=tiles, zoom_start=zoom)    

# amr_metric_community = manip_amr_data_community[['Latitude','Longitude','Transmission score','Access score','Disease score','Use score','Overall score']]
    
    for index, row in user_AMR_env_samp_data.iterrows():
        #print(index, row['village'], row['Latitude'], row['Longitude'])
        html="""
        <h1 style="font-family:arial;text-align:center;">Location: {loca}</h1>
        <p style="font-family:arial;font-size:22px;"><b>Site: {ste}</b></p>
        <p style="{col_pe};font-family:arial;">Prevalence E. coli: {pe:.2f}</p>
        <p style="{col_pa};font-family:arial;">Prevalence AMR: {pa:.2f}</p>
        <p style="{col_pm};font-family:arial;">Prevalence MDR: {pm:.2f}</p>
        """.format(loca=index,ste=row['Site'],\
                   pe=row['Prevalence E. coli'],col_pe=colour_rag_component_env_smp(row['Prevalence E. coli']),\
                   pa=row['Prevalence AMR'],col_pa=colour_rag_component_env_smp(row['Prevalence AMR']),\
                   pm=row['Prevalence MDR'],col_pm=colour_rag_component_env_smp(row['Prevalence MDR']))

        iframe = folium.IFrame(html=html,width=400, height=250)#, width=300, height=300)
        popup = folium.Popup(iframe, parse_html=True, max_width=2650)

        folium.Marker([row['Latitude'],row['Longitude']],popup=popup, tooltip=index, icon = folium.Icon(color='blue',icon='flask',prefix='fa',icon_color='white')).add_to(m)
        # Note - icon colors restricted - see help(folium.Icon), icon_colour accepts HTML - see https://www.rapidtables.com/web/color/html-color-codes.html
        # Note - for icons see https://fontawesome.com/icons?d=gallery&m=free (not all work)
    

    m.add_child(folium.LatLngPopup())#, folium.LayerControl().add_to(m)
    
    fol_pi.Fullscreen(
    position='topright',
    title='Full screen',
    title_cancel='Exit full screen',
    force_separate_button=True).add_to(m)
    
    return m

def create_map_env_sampv2(user_AMR_env_samp_data, lat, lon, tiles="OpenStreetMap", zoom=13):
    m = folium.Map(location=(lat, lon), tiles=tiles, zoom_start=zoom)    

# amr_metric_community = manip_amr_data_community[['Latitude','Longitude','Transmission score','Access score','Disease score','Use score','Overall score']]
    last_row = user_AMR_env_samp_data.index[-1]
    prev_loc = ''
    for index, row in user_AMR_env_samp_data.iterrows():
        # special case for last row
        if index == last_row:
            # check if multi-site
            if row['Location'] == prev_loc:
                html += """
                <p style="font-family:arial;font-size:22px;"><b>Site: {ste}</b></p>
                <p style="{col_pe};font-family:arial;">Prevalence E. coli: {pe:.2f}</p>
                <p style="{col_pa};font-family:arial;">Prevalence AMR: {pa:.2f}</p>
                <p style="{col_pm};font-family:arial;">Prevalence MDR: {pm:.2f}</p>
                """.format(ste=row['Site'],\
                    pe=row['Prevalence E. coli'],col_pe='color: MIDNIGHTBLUE',\
                   pa=row['Prevalence AMR'],col_pa=colour_rag_component_env_smp(row['Prevalence AMR']),\
                   pm=row['Prevalence MDR'],col_pm=colour_rag_component_env_smp(row['Prevalence MDR']))
            
                iframe = folium.IFrame(html=html,width=400, height=250)#, width=300, height=300)
                popup = folium.Popup(iframe, parse_html=True, max_width=2650)
                folium.Marker([row['Latitude'],row['Longitude']],popup=popup, tooltip=row['Location'], icon = folium.Icon(color='blue',icon='flask',prefix='fa',icon_color='white')).add_to(m)
            
            else:
                # not multi-site so store previous html (for df >1 row) and store html of final row
                if last_row != 0:
                            iframe = folium.IFrame(html=html,width=400, height=250)#, width=300, height=300)
                            popup = folium.Popup(iframe, parse_html=True, max_width=2650)

                            folium.Marker([row['Latitude'],row['Longitude']],popup=popup, tooltip=prev_loc, icon = folium.Icon(color='blue',icon='flask',prefix='fa',icon_color='white')).add_to(m)
 
                html = """
                <h1 style="font-family:arial;text-align:center;">Location: {loca}</h1>
                <p style="font-family:arial;font-size:22px;"><b>Site: {ste}</b></p>
                <p style="{col_pe};font-family:arial;">Prevalence E. coli: {pe:.2f}</p>
                <p style="{col_pa};font-family:arial;">Prevalence AMR: {pa:.2f}</p>
                <p style="{col_pm};font-family:arial;">Prevalence MDR: {pm:.2f}</p>
                """.format(loca=row['Location'],ste=row['Site'],\
                   pe=row['Prevalence E. coli'],col_pe='color: MIDNIGHTBLUE',\
                   pa=row['Prevalence AMR'],col_pa=colour_rag_component_env_smp(row['Prevalence AMR']),\
                   pm=row['Prevalence MDR'],col_pm=colour_rag_component_env_smp(row['Prevalence MDR']))
        
                iframe = folium.IFrame(html=html,width=400, height=250)#, width=300, height=300)
                popup = folium.Popup(iframe, parse_html=True, max_width=2650)

                folium.Marker([row['Latitude'],row['Longitude']],popup=popup, tooltip=row['Location'], icon = folium.Icon(color='blue',icon='flask',prefix='fa',icon_color='white')).add_to(m)

    
    
        # special case for first row where df has more than one row:               
        elif index == 0:
            html = """
            <h1 style="font-family:arial;text-align:center;">Location: {loca}</h1>
            <p style="font-family:arial;font-size:22px;"><b>Site: {ste}</b></p>
            <p style="{col_pe};font-family:arial;">Prevalence E. coli: {pe:.2f}</p>
            <p style="{col_pa};font-family:arial;">Prevalence AMR: {pa:.2f}</p>
            <p style="{col_pm};font-family:arial;">Prevalence MDR: {pm:.2f}</p>
            """.format(loca=row['Location'],ste=row['Site'],\
                   pe=row['Prevalence E. coli'],col_pe='color: MIDNIGHTBLUE',\
                   pa=row['Prevalence AMR'],col_pa=colour_rag_component_env_smp(row['Prevalence AMR']),\
                   pm=row['Prevalence MDR'],col_pm=colour_rag_component_env_smp(row['Prevalence MDR']))
            prev_loc = row['Location']
            prev_lat = row['Latitude']
            prev_lon = row['Longitude']
        
        # case for rows inbetween:
        else:
            # where new row is in same 'Location' as previous row
            if row['Location'] == prev_loc:
                html += """
                <p style="font-family:arial;font-size:22px;"><b>Site: {ste}</b></p>
                <p style="{col_pe};font-family:arial;">Prevalence E. coli: {pe:.2f}</p>
                <p style="{col_pa};font-family:arial;">Prevalence AMR: {pa:.2f}</p>
                <p style="{col_pm};font-family:arial;">Prevalence MDR: {pm:.2f}</p>
                """.format(ste=row['Site'],\
                    pe=row['Prevalence E. coli'],col_pe='color: MIDNIGHTBLUE',\
                   pa=row['Prevalence AMR'],col_pa=colour_rag_component_env_smp(row['Prevalence AMR']),\
                   pm=row['Prevalence MDR'],col_pm=colour_rag_component_env_smp(row['Prevalence MDR']))

            # where new row is NOT in same 'Location' as previous row:
            else:
                #first store previous html and lat lon
                iframe = folium.IFrame(html=html,width=400, height=250)#, width=300, height=300)
                popup = folium.Popup(iframe, parse_html=True, max_width=2650)

                folium.Marker([prev_lat,prev_lon],popup=popup, tooltip=prev_loc, icon = folium.Icon(color='blue',icon='flask',prefix='fa',icon_color='white')).add_to(m)
                #next generate new html and update loc/lat/lon
                html = """
                <h1 style="font-family:arial;text-align:center;">Location: {loca}</h1>
                <p style="font-family:arial;font-size:22px;"><b>Site: {ste}</b></p>
                <p style="{col_pe};font-family:arial;">Prevalence E. coli: {pe:.2f}</p>
                <p style="{col_pa};font-family:arial;">Prevalence AMR: {pa:.2f}</p>
                <p style="{col_pm};font-family:arial;">Prevalence MDR: {pm:.2f}</p>
                """.format(loca=row['Location'],ste=row['Site'],\
                   pe=row['Prevalence E. coli'],col_pe='color: MIDNIGHTBLUE',\
                   pa=row['Prevalence AMR'],col_pa=colour_rag_component_env_smp(row['Prevalence AMR']),\
                   pm=row['Prevalence MDR'],col_pm=colour_rag_component_env_smp(row['Prevalence MDR']))
                prev_loc = row['Location']
                prev_lat = row['Latitude']
                prev_lon = row['Longitude']
    
        # Note - icon colors restricted - see help(folium.Icon), icon_colour accepts HTML - see https://www.rapidtables.com/web/color/html-color-codes.html
        # Note - for icons see https://fontawesome.com/icons?d=gallery&m=free (not all work)
    

    m.add_child(folium.LatLngPopup())#, folium.LayerControl().add_to(m)
    
    fol_pi.Fullscreen(
    position='topright',
    title='Full screen',
    title_cancel='Exit full screen',
    force_separate_button=True).add_to(m)
    
    return m

def create_map_env_sampv3(user_AMR_env_samp_data, m):
    last_row = user_AMR_env_samp_data.index[-1]
    prev_loc = ''
    for index, row in user_AMR_env_samp_data.iterrows():
        # special case for last row
        if index == last_row:
            # check if multi-site
            if row['Location'] == prev_loc:
                html += """
                <p style="font-family:arial;font-size:22px;"><b>Site: {ste}</b></p>
                <p style="{col_pe};font-family:arial;">Prevalence E. coli: {pe:.2f}</p>
                <p style="{col_pa};font-family:arial;">Prevalence AMR: {pa:.2f}</p>
                <p style="{col_pm};font-family:arial;">Prevalence MDR: {pm:.2f}</p>
                """.format(ste=row['Site'],\
                    pe=row['Prevalence E. coli'],col_pe='color: MIDNIGHTBLUE',\
                   pa=row['Prevalence AMR'],col_pa=colour_rag_component_env_smp(row['Prevalence AMR']),\
                   pm=row['Prevalence MDR'],col_pm=colour_rag_component_env_smp(row['Prevalence MDR']))
            
                iframe = folium.IFrame(html=html,width=400, height=250)#, width=300, height=300)
                popup = folium.Popup(iframe, parse_html=True, max_width=2650)
                folium.Marker([row['Latitude'],row['Longitude']],popup=popup, tooltip=row['Location'], icon = folium.Icon(color='blue',icon='flask',prefix='fa',icon_color='white')).add_to(m)
            
            else:
                # not multi-site so store previous html (for df >1 row) and store html of final row
                if last_row != 0:
                            iframe = folium.IFrame(html=html,width=400, height=250)#, width=300, height=300)
                            popup = folium.Popup(iframe, parse_html=True, max_width=2650)

                            folium.Marker([row['Latitude'],row['Longitude']],popup=popup, tooltip=prev_loc, icon = folium.Icon(color='blue',icon='flask',prefix='fa',icon_color='white')).add_to(m)
 
                html = """
                <h1 style="font-family:arial;text-align:center;">Location: {loca}</h1>
                <p style="font-family:arial;font-size:22px;"><b>Site: {ste}</b></p>
                <p style="{col_pe};font-family:arial;">Prevalence E. coli: {pe:.2f}</p>
                <p style="{col_pa};font-family:arial;">Prevalence AMR: {pa:.2f}</p>
                <p style="{col_pm};font-family:arial;">Prevalence MDR: {pm:.2f}</p>
                """.format(loca=row['Location'],ste=row['Site'],\
                   pe=row['Prevalence E. coli'],col_pe='color: MIDNIGHTBLUE',\
                   pa=row['Prevalence AMR'],col_pa=colour_rag_component_env_smp(row['Prevalence AMR']),\
                   pm=row['Prevalence MDR'],col_pm=colour_rag_component_env_smp(row['Prevalence MDR']))
        
                iframe = folium.IFrame(html=html,width=400, height=250)#, width=300, height=300)
                popup = folium.Popup(iframe, parse_html=True, max_width=2650)

                folium.Marker([row['Latitude'],row['Longitude']],popup=popup, tooltip=row['Location'], icon = folium.Icon(color='blue',icon='flask',prefix='fa',icon_color='white')).add_to(m)

    
    
        # special case for first row where df has more than one row:               
        elif index == 0:
            html = """
            <h1 style="font-family:arial;text-align:center;">Location: {loca}</h1>
            <p style="font-family:arial;font-size:22px;"><b>Site: {ste}</b></p>
            <p style="{col_pe};font-family:arial;">Prevalence E. coli: {pe:.2f}</p>
            <p style="{col_pa};font-family:arial;">Prevalence AMR: {pa:.2f}</p>
            <p style="{col_pm};font-family:arial;">Prevalence MDR: {pm:.2f}</p>
            """.format(loca=row['Location'],ste=row['Site'],\
                   pe=row['Prevalence E. coli'],col_pe='color: MIDNIGHTBLUE',\
                   pa=row['Prevalence AMR'],col_pa=colour_rag_component_env_smp(row['Prevalence AMR']),\
                   pm=row['Prevalence MDR'],col_pm=colour_rag_component_env_smp(row['Prevalence MDR']))
            prev_loc = row['Location']
            prev_lat = row['Latitude']
            prev_lon = row['Longitude']
        
        # case for rows inbetween:
        else:
            # where new row is in same 'Location' as previous row
            if row['Location'] == prev_loc:
                html += """
                <p style="font-family:arial;font-size:22px;"><b>Site: {ste}</b></p>
                <p style="{col_pe};font-family:arial;">Prevalence E. coli: {pe:.2f}</p>
                <p style="{col_pa};font-family:arial;">Prevalence AMR: {pa:.2f}</p>
                <p style="{col_pm};font-family:arial;">Prevalence MDR: {pm:.2f}</p>
                """.format(ste=row['Site'],\
                    pe=row['Prevalence E. coli'],col_pe='color: MIDNIGHTBLUE',\
                   pa=row['Prevalence AMR'],col_pa=colour_rag_component_env_smp(row['Prevalence AMR']),\
                   pm=row['Prevalence MDR'],col_pm=colour_rag_component_env_smp(row['Prevalence MDR']))

            # where new row is NOT in same 'Location' as previous row:
            else:
                #first store previous html and lat lon
                iframe = folium.IFrame(html=html,width=400, height=250)#, width=300, height=300)
                popup = folium.Popup(iframe, parse_html=True, max_width=2650)

                folium.Marker([prev_lat,prev_lon],popup=popup, tooltip=prev_loc, icon = folium.Icon(color='blue',icon='flask',prefix='fa',icon_color='white')).add_to(m)
                #next generate new html and update loc/lat/lon
                html = """
                <h1 style="font-family:arial;text-align:center;">Location: {loca}</h1>
                <p style="font-family:arial;font-size:22px;"><b>Site: {ste}</b></p>
                <p style="{col_pe};font-family:arial;">Prevalence E. coli: {pe:.2f}</p>
                <p style="{col_pa};font-family:arial;">Prevalence AMR: {pa:.2f}</p>
                <p style="{col_pm};font-family:arial;">Prevalence MDR: {pm:.2f}</p>
                """.format(loca=row['Location'],ste=row['Site'],\
                   pe=row['Prevalence E. coli'],col_pe='color: MIDNIGHTBLUE',\
                   pa=row['Prevalence AMR'],col_pa=colour_rag_component_env_smp(row['Prevalence AMR']),\
                   pm=row['Prevalence MDR'],col_pm=colour_rag_component_env_smp(row['Prevalence MDR']))
                prev_loc = row['Location']
                prev_lat = row['Latitude']
                prev_lon = row['Longitude']
    
        # Note - icon colors restricted - see help(folium.Icon), icon_colour accepts HTML - see https://www.rapidtables.com/web/color/html-color-codes.html
        # Note - for icons see https://fontawesome.com/icons?d=gallery&m=free (not all work)
    return(m)

def look_tab(w='',v='AMR metric scores', u='Community level', input_data=''):
    if w=='Please select community:': return  IPdisp.display(widgets.HTML(value="<p style='font-size:14px;text-align:center;color:LIGHTGRAY;'>Data displayed here once AMR Metric community selected</p>"))
    if w=='Select AMR Metric region above to view data': return IPdisp.display(widgets.HTML(value="<p style='font-size:14px;text-align:center;color:LIGHTGRAY;'>Data displayed here once AMR Metric region and community selected</p>"))
    if w=='All':
        if v == 'All features':
            if u == 'Community level':
                display_data = base_AMR_data.groupby(['Community']).mean().drop(columns=['ID']).copy()
                #Convert disease score to community risk level
                display_data[['Livestock_1 Disease priority','Livestock_2 Disease priority','Livestock_3 Disease priority']] = display_data[['Livestock_1 Disease priority','Livestock_2 Disease priority','Livestock_3 Disease priority']].applymap(lambda x: displ_risk_comm(x))
               
            if u == 'Household level':
                display_data = base_AMR_data.copy()
                #Convert 1 and 0 to Yes and No
                display_data[['Transmission_A','Transmission_B','Transmission_C',\
                              'Use of trained animal healthcare professional',\
                              'Mixing antibiotics in the same animal','Not completing course',\
                              'Not measuring dose accurately','Using for prevention','Drink milk from treated animals',\
                              'Eat meat from treated animals']] = display_data[['Transmission_A','Transmission_B','Transmission_C',\
                                                                                'Use of trained animal healthcare professional','Mixing antibiotics in the same animal',\
                                                                                'Not completing course','Not measuring dose accurately','Using for prevention',\
                                                                                'Drink milk from treated animals','Eat meat from treated animals']].applymap(lambda x: displ_YN(x))    
                #Convert disease score to risk level
                display_data[['Livestock_1 Disease priority','Livestock_2 Disease priority','Livestock_3 Disease priority']] = display_data[['Livestock_1 Disease priority','Livestock_2 Disease priority','Livestock_3 Disease priority']].applymap(lambda x: displ_risk(x))    
            
            if input_data =='Gujarat, India':
                display_data.rename(columns = {"Transmission_A":"Spreading dung on fields",
                                           "Transmission_B":"Not cleaning with disinfectant",
                                           "Transmission_C":"Visited by animal healthcare worker",
                                           "Livestock_1 herd size":"Cattle herd size",
                                           "Livestock_2 herd size":"Buffalo herd size",
                                           "Livestock_3 herd size":"Goat herd size",
                                           "Livestock_1 Disease priority":"Disease priority cattle",
                                           "Livestock_2 Disease priority":"Disease priority buffalo",
                                           "Livestock_3 Disease priority":"Disease priority goats"}, 
                                 inplace = True) 
            if input_data =='Pune, India':
                display_data.rename(columns = {"Transmission_A":"Spreading dung on fields",
                                           "Transmission_B":"Not cleaning with disinfectant",
                                           "Transmission_C":"Visited by animal healthcare worker",
                                           "Livestock_1 herd size":"Cattle herd size",
                                           "Livestock_2 herd size":"Buffalo herd size",
                                           "Livestock_3 herd size":"Chicken flock size",
                                           "Livestock_1 Disease priority":"Disease priority cattle",
                                           "Livestock_2 Disease priority":"Disease priority buffalo",
                                           "Livestock_3 Disease priority":"Disease priority chickens"}, 
                                 inplace = True)
            if input_data in ('Kajiado, Kenya','Kajiado, Kenya incl. Environmental Samples'):
                display_data.rename(columns = {"Transmission_A":"Use of communal water source",
                                           "Transmission_B":"Use of livestock market",
                                           "Transmission_C":"Use of abattoir",
                                           "Livestock_1 herd size":"Cattle herd size",
                                           "Livestock_2 herd size":"Sheep herd size",
                                           "Livestock_3 herd size":"Goat herd size",
                                           "Livestock_1 Disease priority":"Disease priority cattle",
                                           "Livestock_2 Disease priority":"Disease priority sheep",
                                           "Livestock_3 Disease priority":"Disease priority goats"}, 
                                 inplace = True)                
            if input_data =='Kiambu, Kenya':
                display_data.rename(columns = {"Transmission_A":"Use of communal water source",
                                           "Transmission_B":"Use of livestock market",
                                           "Transmission_C":"Use of abattoir",
                                           "Livestock_1 herd size":"Cattle herd size",
                                           "Livestock_2 herd size":"Sheep flock size",
                                           "Livestock_3 herd size":"Chicken flock size",
                                           "Livestock_1 Disease priority":"Disease priority cattle",
                                           "Livestock_2 Disease priority":"Disease priority sheep",
                                           "Livestock_3 Disease priority":"Disease priority chickens"}, 
                                 inplace = True)     
            
            
        if v == 'AMR metric scores':
            if u == 'Community level':
                display_data = base_AMR_data.groupby(['Community']).mean()[['Latitude','Longitude','Transmission score','Access score','Disease score','Use score','Overall score']].copy()
            if u == 'Household level':
                display_data = base_AMR_data[['ID','Community','Latitude','Longitude','Transmission score','Access score','Disease score','Use score','Overall score']].copy()
            
    else: # For specific community selection ny user (set as w)
        if v == 'All features':
            if u == 'Community level':
                display_data = base_AMR_data.groupby(['Community']).mean().loc[[w]].drop(columns=['ID']).copy()
                #Convert disease score to risk level
                display_data[['Livestock_1 Disease priority','Livestock_2 Disease priority','Livestock_3 Disease priority']] = display_data[['Livestock_1 Disease priority','Livestock_2 Disease priority','Livestock_3 Disease priority']].applymap(lambda x: displ_risk_comm(x))
                

            if u == 'Household level':
                display_data = base_AMR_data[base_AMR_data['Community']==w].copy()
                #Convert 1 and 0 to Yes and No
                display_data[['Transmission_A','Transmission_B','Transmission_C',\
                              'Use of trained animal healthcare professional',\
                              'Mixing antibiotics in the same animal','Not completing course',\
                              'Not measuring dose accurately','Using for prevention','Drink milk from treated animals',\
                              'Eat meat from treated animals']] = display_data[['Transmission_A','Transmission_B','Transmission_C',\
                                                                                'Use of trained animal healthcare professional','Mixing antibiotics in the same animal',\
                                                                                'Not completing course','Not measuring dose accurately','Using for prevention',\
                                                                                'Drink milk from treated animals','Eat meat from treated animals']].applymap(lambda x: displ_YN(x))
                #Convert disease score to risk level
                display_data[['Livestock_1 Disease priority','Livestock_2 Disease priority','Livestock_3 Disease priority']] = display_data[['Livestock_1 Disease priority','Livestock_2 Disease priority','Livestock_3 Disease priority']].applymap(lambda x: displ_risk(x))    

            if input_data =='Gujarat, India':
                display_data.rename(columns = {"Transmission_A":"Spreading dung on fields",
                                           "Transmission_B":"Not cleaning with disinfectant",
                                           "Transmission_C":"Visited by animal healthcare worker",
                                           "Livestock_1 herd size":"Cattle herd size",
                                           "Livestock_2 herd size":"Buffalo herd size",
                                           "Livestock_3 herd size":"Goat herd size",
                                           "Livestock_1 Disease priority":"Disease priority cattle",
                                           "Livestock_2 Disease priority":"Disease priority buffalo",
                                           "Livestock_3 Disease priority":"Disease priority goats"}, 
                                 inplace = True) 
            if input_data =='Pune, India':
                display_data.rename(columns = {"Transmission_A":"Spreading dung on fields",
                                           "Transmission_B":"Not cleaning with disinfectant",
                                           "Transmission_C":"Visited by animal healthcare worker",
                                           "Livestock_1 herd size":"Cattle herd size",
                                           "Livestock_2 herd size":"Buffalo herd size",
                                           "Livestock_3 herd size":"Chicken flock size",
                                           "Livestock_1 Disease priority":"Disease priority cattle",
                                           "Livestock_2 Disease priority":"Disease priority buffalo",
                                           "Livestock_3 Disease priority":"Disease priority chickens"}, 
                                 inplace = True)
            if input_data in ('Kajiado, Kenya','Kajiado, Kenya incl. Environmental Samples'):
                display_data.rename(columns = {"Transmission_A":"Use of communal water source",
                                           "Transmission_B":"Use of livestock market",
                                           "Transmission_C":"Use of abattoir",
                                           "Livestock_1 herd size":"Cattle herd size",
                                           "Livestock_2 herd size":"Sheep flock size",
                                           "Livestock_3 herd size":"Goat herd size",
                                           "Livestock_1 Disease priority":"Disease priority cattle",
                                           "Livestock_2 Disease priority":"Disease priority sheep",
                                           "Livestock_3 Disease priority":"Disease priority goats"}, 
                                 inplace = True)
            if input_data =='Kiambu, Kenya':
                display_data.rename(columns = {"Transmission_A":"Use of communal water source",
                                           "Transmission_B":"Use of livestock market",
                                           "Transmission_C":"Use of abattoir",
                                           "Livestock_1 herd size":"Cattle herd size",
                                           "Livestock_2 herd size":"Sheep flock size",
                                           "Livestock_3 herd size":"Chicken flock size",
                                           "Livestock_1 Disease priority":"Disease priority cattle",
                                           "Livestock_2 Disease priority":"Disease priority sheep",
                                           "Livestock_3 Disease priority":"Disease priority chickens"}, 
                                 inplace = True)
                
        if v == 'AMR metric scores':
            if u == 'Community level':
                display_data = base_AMR_data.groupby(['Community']).mean().loc[[w]][['Latitude','Longitude','Transmission score','Access score','Disease score','Use score','Overall score']].copy()# amr_data[amr_data['village']==w].loc[amr_data['indicator'] == 'M']
            if u == 'Household level':
                display_data = base_AMR_data[base_AMR_data['Community']==w][['Community','Latitude','Longitude','Transmission score','Access score','Disease score','Use score','Overall score']].copy()
    
    if v == 'All features' and u == 'Community level':
        return IPdisp.display(display_data\
                          .style.applymap(colour_rag_component, subset = ['Transmission score','Access score','Disease score','Use score'])\
                         .applymap(colour_rag_overall, subset = 'Overall score')\
                              .set_table_styles(magnify2())
                             .format({'Spreading dung on fields': "{:.2%}",'Not cleaning with disinfectant': "{:.2%}",'Visited by animal healthcare worker': "{:.2%}",\
                                      'Use of communal water source': "{:.2%}",'Use of livestock market': "{:.2%}",'Use of abattoir': "{:.2%}",\
                                           'Use of trained animal healthcare professional': "{:.2%}",\
                                          'Mixing antibiotics in the same animal': "{:.2%}",'Not completing course': "{:.2%}",\
                                          'Not measuring dose accurately': "{:.2%}",'Using for prevention': "{:.2%}",\
                                          'Drink milk from treated animals': "{:.2%}",'Eat meat from treated animals': "{:.2%}"}))
    else:
        return IPdisp.display(display_data\
                          .style.applymap(colour_rag_component, subset = ['Transmission score','Access score','Disease score','Use score'])\
                         .applymap(colour_rag_overall, subset = 'Overall score')\
                              .set_table_styles(magnify2()))
                         

def look_manip(w='',input_data='', add_dung_widg = 0, add_clean_widg = 0, add_visit_widg = 0,\
             add_freq_widg=0, add_trained_widg=0,\
            add_c_herd_widg=0, add_b_herd_widg=0, add_g_herd_widg=0,\
             add_c_dis_widg=0, add_b_dis_widg=0, add_g_dis_widg=0,\
             add_mix_anti_widg=0, add_not_comp_widg=0, add_not_meas_widg=0,\
             add_use_prev_widg=0, add_drink_mlk_widg=0, add_eat_meat_widg=0):
    if w=='Select AMR Metric region above to view data': return IPdisp.display(widgets.HTML(value="<p style='font-size:14px;text-align:center;color:LIGHTGRAY;'>Data displayed here once AMR Metric region selected</p>"))
    #elif w=='Kajiado, Kenya Environment Samples': return IPdisp.display(widgets.HTML(value="<p style='font-size:14px;text-align:center;color:LIGHTGRAY;'>Data displayed here once AMR Metric region selected</p>"))
    else:
        return IPdisp.display(user_AMR_data[['Latitude','Longitude','Transmission score','Access score','Disease score','Use score','Overall score']]\
                          .style.applymap(colour_rag_component, subset = ['Transmission score','Access score','Disease score','Use score'])\
                         .applymap(colour_rag_overall, subset = 'Overall score')\
                         .set_table_styles(magnify2()))
    
def look_tab_samp(loc_list='', input_data=''):
    if input_data not in ('Kajiado, Kenya Environmental Samples','Kajiado, Kenya incl. Environmental Samples'): return  IPdisp.display(widgets.HTML(value="<p style='font-size:14px;text-align:center;color:LIGHTGRAY;'>Data displayed here once Environmental Sample region selected</p>"))
    if input_data in ('Kajiado, Kenya Environmental Samples','Kajiado, Kenya incl. Environmental Samples'):
        if loc_list == 'Please select location:': return  IPdisp.display(widgets.HTML(value="<p style='font-size:14px;text-align:center;color:LIGHTGRAY;'>Data displayed here once Environmental Sample region and location selected</p>"))
        elif loc_list == 'All':
            return IPdisp.display(env_sample_data_full[['Location','Site','Sample type','E. coli present',\
                                                        'AMR present','MDR present','Ceftriaxone','Tetracycline','Ciprofloxacin','Ampicillin','Chloramphenicol','Gentamycin','Streptomycin','Trimethoprim']].fillna(value='')\
                                 .style.applymap(colour_rag_table_env, subset = ['AMR present','MDR present','Ceftriaxone','Tetracycline','Ciprofloxacin','Ampicillin','Chloramphenicol','Gentamycin','Streptomycin','Trimethoprim'])\
                                 .set_table_styles(magnify2()))
        else:
            return IPdisp.display(env_sample_data_full[env_sample_data_full['Location']==loc_list][['Site','Sample type','E. coli present',\
                                                        'AMR present','MDR present','Ceftriaxone','Tetracycline','Ciprofloxacin','Ampicillin','Chloramphenicol','Gentamycin','Streptomycin','Trimethoprim']].fillna(value='')\
                                 .style.applymap(colour_rag_table_env, subset = ['AMR present','MDR present','Ceftriaxone','Tetracycline','Ciprofloxacin','Ampicillin','Chloramphenicol','Gentamycin','Streptomycin','Trimethoprim'])\
                                 .set_table_styles(magnify2()))

In [8]:
#sample_locations = ['Please select region to display map and data','Gujarat, India', 'Pune, India', 'Kajiado, Kenya','Kajiado, Kenya Environmental Samples','Kajiado, Kenya incl. Environmental Samples','Kiambu, Kenya']
sample_locations = ['Please select region to display map and data','Gujarat, India', 'Pune, India','Kajiado, Kenya incl. Environmental Samples','Kiambu, Kenya']
look_item = ['AMR metric scores','All features']
gran_lev = ['Community level','Household level']

def AMR_tool(input_data='', add_dung_widg = 0, add_clean_widg = 0, add_visit_widg = 0,\
             add_freq_widg=0, add_trained_widg=0,\
            add_c_herd_widg=0, add_b_herd_widg=0, add_g_herd_widg=0,\
             add_c_dis_widg=0, add_b_dis_widg=0, add_g_dis_widg=0,\
             add_mix_anti_widg=0, add_not_comp_widg=0, add_not_meas_widg=0,\
             add_use_prev_widg=0, add_drink_mlk_widg=0, add_eat_meat_widg=0):

    global base_AMR_data
    global community_list
    global location_list
    global user_AMR_data
    global env_sample_data_full

    
    if input_data == 'Gujarat, India':
        amr_input_data = pd.read_csv("data/gujarat_csv_data_simplified.csv") #load csv

        base_AMR_data = base_AMR_metr(amr_input_data) #calculate base AMR metric on all households
        
        lat_mean = (base_AMR_data['Latitude'].min() + base_AMR_data['Latitude'].max())/2
        lon_mean = (base_AMR_data['Longitude'].min() + base_AMR_data['Longitude'].max())/2
        
        community_list = ['Please select community:']+['All']+sorted(base_AMR_data['Community'].unique().tolist())
        location_list = ['Select Environmental Sample region to view data']
        region_zoom = 13
    elif input_data == 'Pune, India':
        amr_input_data = pd.read_csv("data/pune_csv_data_simplified.csv") #load csv

        base_AMR_data = base_AMR_metr(amr_input_data) #calculate base AMR metric on all households
        
        lat_mean = (base_AMR_data['Latitude'].min() + base_AMR_data['Latitude'].max())/2
        lon_mean = (base_AMR_data['Longitude'].min() + base_AMR_data['Longitude'].max())/2
        
        community_list = ['Please select community:']+['All']+sorted(base_AMR_data['Community'].unique().tolist())
        location_list = ['Select Environmental Sample region to view data']
        region_zoom = 10
    elif input_data == 'Kajiado, Kenya':
        amr_input_data = pd.read_csv("data/kajiado_csv_data_simplified.csv") #load csv

        base_AMR_data = base_AMR_metr(amr_input_data) #calculate base AMR metric on all households
        
        lat_mean = (base_AMR_data['Latitude'].min() + base_AMR_data['Latitude'].max())/2
        lon_mean = (base_AMR_data['Longitude'].min() + base_AMR_data['Longitude'].max())/2
        
        community_list = ['Please select community:']+['All']+sorted(base_AMR_data['Community'].unique().tolist())
        location_list = ['Select Environmental Sample region to view data']
        region_zoom = 10
        
    elif input_data == 'Kiambu, Kenya':
        amr_input_data = pd.read_csv("data/kiambu_csv_data_simplified.csv") #load csv
        
        base_AMR_data = base_AMR_metr(amr_input_data) #calculate base AMR metric on all households
        
        lat_mean = (base_AMR_data['Latitude'].min() + base_AMR_data['Latitude'].max())/2
        lon_mean = (base_AMR_data['Longitude'].min() + base_AMR_data['Longitude'].max())/2
        
        community_list = ['Please select community:']+['All']+sorted(base_AMR_data['Community'].unique().tolist())
        location_list = ['Select Environmental Sample region to view data']
        region_zoom = 13
    
    elif input_data == 'Kajiado, Kenya Environmental Samples':
        amr_input_data = pd.read_csv("data/Final_Kajiado_Environmental_sampling_data.csv") #load csv

        base_AMR_data = [0]
        
        env_samp_map_data = amr_input_data[['Latitude','Longitude','Location','Site','Prevalence E. coli','Prevalence AMR','Prevalence MDR']].groupby(['Location','Site'], as_index=False).mean()
        #env_samp_map_data.set_index(['Location'])
        #base_AMR_data = base_AMR_metr(amr_input_data) #calculate base AMR metric on all households
        
        lat_mean = (env_samp_map_data['Latitude'].min() + env_samp_map_data['Latitude'].max())/2
        lon_mean = (env_samp_map_data['Longitude'].min() + env_samp_map_data['Longitude'].max())/2
        
        community_list = ['Select AMR Metric region above to view data']
        location_list = ['Please select location:']+['All']+sorted(amr_input_data['Location'].unique().tolist())        
        
        region_zoom = 10
        # show environment map
        m = create_map_env_sampv2(user_AMR_env_samp_data=env_samp_map_data, lat=lat_mean, lon=lon_mean, tiles="OpenStreetMap", zoom=region_zoom)
        
        env_sample_data_full = amr_input_data
        
        return IPdisp.display(m)

    # Combination
    elif input_data == 'Kajiado, Kenya incl. Environmental Samples':
        
        amr_input_data_env = pd.read_csv("data/Final_Kajiado_Environmental_sampling_data.csv") #load csv
        amr_input_data_amr = pd.read_csv("data/kajiado_csv_data_simplified.csv") #load csv

        base_AMR_data = base_AMR_metr(amr_input_data_amr) #calculate base AMR metric on all households
        
        lat_mean = (base_AMR_data['Latitude'].min() + base_AMR_data['Latitude'].max())/2
        lon_mean = (base_AMR_data['Longitude'].min() + base_AMR_data['Longitude'].max())/2
        
        community_list = ['Please select community:']+['All']+sorted(base_AMR_data['Community'].unique().tolist())

        region_zoom = 10
        
        env_samp_map_data = amr_input_data_env[['Latitude','Longitude','Location','Site','Prevalence E. coli','Prevalence AMR','Prevalence MDR']].groupby(['Location','Site'], as_index=False).mean()
        #env_samp_map_data.set_index(['Location'])
        #base_AMR_data = base_AMR_metr(amr_input_data) #calculate base AMR metric on all households

        
        location_list = ['Please select location:']+['All']+sorted(amr_input_data_env['Location'].unique().tolist())        
        

        env_sample_data_full = amr_input_data_env
        
        

        user_AMR_data = user_AMR_comp(amr_input_data_raw=amr_input_data_amr,\
                                  add_dung=add_dung_widg, add_clean=add_clean_widg, add_visit=add_visit_widg,\
                                  add_freq=add_freq_widg, add_trained=add_trained_widg,\
                                  add_c_herd = add_c_herd_widg, add_b_herd = add_b_herd_widg, add_g_herd = add_g_herd_widg,\
                                  add_c_dis = add_c_dis_widg, add_b_dis = add_b_dis_widg, add_g_dis = add_g_dis_widg,\
                                  add_mix_anti = add_mix_anti_widg, add_not_comp = add_not_comp_widg, add_not_meas = add_not_meas_widg,\
                                  add_use_prev = add_use_prev_widg, add_drink_mlk = add_drink_mlk_widg, add_eat_meat = add_eat_meat_widg)
        
        # Show user manipulated data on map
        m_amr = create_map(user_AMR_data=user_AMR_data, lat=lat_mean, lon=lon_mean, tiles="OpenStreetMap", zoom=region_zoom)
        
        m_amr_env = create_map_env_sampv3(env_samp_map_data, m_amr)
        return IPdisp.display(m_amr_env)
    
    else:
        base_AMR_data = [0]
        
        community_list = ['Select AMR Metric region above to view data']
        location_list = ['Select Environmental Sample region to view data']
        return
        
    # Generate user manipulated data
    user_AMR_data = user_AMR_comp(amr_input_data_raw=amr_input_data,\
                                  add_dung=add_dung_widg, add_clean=add_clean_widg, add_visit=add_visit_widg,\
                                  add_freq=add_freq_widg, add_trained=add_trained_widg,\
                                  add_c_herd = add_c_herd_widg, add_b_herd = add_b_herd_widg, add_g_herd = add_g_herd_widg,\
                                  add_c_dis = add_c_dis_widg, add_b_dis = add_b_dis_widg, add_g_dis = add_g_dis_widg,\
                                  add_mix_anti = add_mix_anti_widg, add_not_comp = add_not_comp_widg, add_not_meas = add_not_meas_widg,\
                                  add_use_prev = add_use_prev_widg, add_drink_mlk = add_drink_mlk_widg, add_eat_meat = add_eat_meat_widg)
        
    # Show user manipulated data on map
    m = create_map(user_AMR_data=user_AMR_data, lat=lat_mean, lon=lon_mean, tiles="OpenStreetMap", zoom=region_zoom)
    
    
    return IPdisp.display(m)

# Build map widgets

title = widgets.HTML(value="<h1 style='color:WHITE;font-size:40px;background-color:MIDNIGHTBLUE;text-align:center;border:2px solid Black;'>\
    AMR Data Exploration Tool <a style='color:LIGHTSEAGREEN;font-size:20px;'> v2.0</a></h1> \
    <p style='font-size:20px;text-align:center;'>A tool that allows the user to view and manipulate AMR data for regions in Africa and India</p>",\
                     layout=widgets.Layout(width='100%', height='200%'))


input_data = widgets.Dropdown(options=sample_locations)

add_dung_widg = widgets.FloatSlider(value=0,min=-1,max=1,step=0.05,description='Spreading dung on fields:',readout_format='+.0%',continuous_update=False)
# The below will change Trnasmission A widget description based on whether India or Africa transmission variables
def transmission_A_change(change):
    if change.new in ('Gujarat, India','Pune, India'):
        add_dung_widg.description = 'Spreading dung on fields:'
    else:
        add_dung_widg.description = 'Use of communal water source:'
input_data.observe(transmission_A_change, names='value')

# The below will change Trnasmission B widget description based on whether India or Africa transmission variables
add_clean_widg = widgets.FloatSlider(value=0,min=-1,max=1,step=0.05,description='Not cleaning shed with disinfectant:',readout_format='+.0%',continuous_update=False)
def transmission_B_change(change):
    if change.new in ('Gujarat, India','Pune, India'):
        add_clean_widg.description = 'Not cleaning shed with disinfectant:'
    else:
        add_clean_widg.description = 'Use of livestock market'
input_data.observe(transmission_B_change, names='value')

# The below will change Trnasmission C widget description based on whether India or Africa transmission variables
add_visit_widg = widgets.FloatSlider(value=0,min=-1,max=1,step=0.05,description='Visited by animal healthcare worker:',readout_format='+.0%',continuous_update=False)
def transmission_C_change(change):
    if change.new in ('Gujarat, India','Pune, India'):
        add_visit_widg.description = 'Visited by animal healthcare worker:'
    else:
        add_visit_widg.description = 'Use of abattoir'
input_data.observe(transmission_C_change, names='value')

add_trained_widg = widgets.FloatSlider(value=0,min=-2,max=2,step=1,description='Use of trained animal healthcare professional:',readout_format='+d',continuous_update=False)
add_freq_widg = widgets.BoundedIntText(value=+0,min=-100,max=100,step=1,description='+/- Frequency of antibiotic purchase:',continuous_update=False)

add_c_herd_widg = widgets.BoundedIntText(value=0,min=-100,max=100,step=10,description='+/- Cattle herd size:',continuous_update=False,style = {'description_width': 'initial'})
# The below will change add_c_herd_widg widget description based on which region is selected
def herd_size_1_change(change):
    if change.new == 'Gujarat, India':
        add_c_herd_widg.description = '+/- Cattle herd size:'
    elif change.new == 'Pune, India':
        add_c_herd_widg.description = '+/- Cattle herd size:'
    elif change.new in ('Kajiado, Kenya','Kajiado, Kenya incl. Environmental Samples'):
        add_c_herd_widg.description = '+/- Cattle herd size:'
    elif change.new == 'Kiambu, Kenya':
        add_c_herd_widg.description = '+/- Cattle herd size:'        
input_data.observe(herd_size_1_change, names='value')

add_b_herd_widg = widgets.BoundedIntText(value=0,min=-100,max=100,step=10,description='+/- Buffalo herd size:',continuous_update=False,style = {'description_width': 'initial'})
# The below will change add_b_herd_widg widget description based on which region is selected
def herd_size_2_change(change):
    if change.new == 'Gujarat, India':
        add_b_herd_widg.description = '+/- Buffalo herd size:'
    elif change.new == 'Pune, India':
        add_b_herd_widg.description = '+/- Buffalo herd size:'
    elif change.new in ('Kajiado, Kenya','Kajiado, Kenya incl. Environmental Samples'):
        add_b_herd_widg.description = '+/- Sheep flock size:'
    elif change.new == 'Kiambu, Kenya':
        add_b_herd_widg.description = '+/- Sheep flock size:'        
input_data.observe(herd_size_2_change, names='value')

add_g_herd_widg = widgets.BoundedIntText(value=0,min=-100,max=100,step=10,description='+/- Goat herd size:',continuous_update=False,style = {'description_width': 'initial'})
# The below will change add_g_herd_widg widget description based on which region is selected
def herd_size_3_change(change):
    if change.new == 'Gujarat, India':
        add_g_herd_widg.description = '+/- Goat herd size:'
    elif change.new == 'Pune, India':
        add_g_herd_widg.description = '+/- Chicken flock size:'
    elif change.new in ('Kajiado, Kenya','Kajiado, Kenya incl. Environmental Samples'):
        add_g_herd_widg.description = '+/- Goat herd size:'
    elif change.new == 'Kiambu, Kenya':
        add_g_herd_widg.description = '+/- Chicken flock size:'        
input_data.observe(herd_size_3_change, names='value')


add_c_dis_widg = widgets.FloatSlider(value=0,min=-2,max=2,step=1,description='Cattle disease risk household level:',readout_format='+d',continuous_update=False)
# The below will change add_c_dis_widg widget description based on which region is selected
def dis_pri_1_change(change):
    if change.new == 'Gujarat, India':
        add_c_dis_widg.description = 'Cattle disease risk household level:'
    elif change.new == 'Pune, India':
        add_c_dis_widg.description = 'Cattle disease risk household level:'
    elif change.new in ('Kajiado, Kenya','Kajiado, Kenya incl. Environmental Samples'):
        add_c_dis_widg.description = 'Cattle disease risk household level:'
    elif change.new == 'Kiambu, Kenya':
        add_c_dis_widg.description = 'Cattle disease risk household level:'        
input_data.observe(dis_pri_1_change, names='value')

add_b_dis_widg = widgets.FloatSlider(value=0,min=-2,max=2,step=1,description='Buffalo disease risk household level:',readout_format='+d',continuous_update=False)
# The below will change add_b_dis_widg widget description based on which region is selected
def dis_pri_2_change(change):
    if change.new == 'Gujarat, India':
        add_b_dis_widg.description = 'Buffalo disease risk household level:'
    elif change.new == 'Pune, India':
        add_b_dis_widg.description = 'Buffalo disease risk household level:'
    elif change.new in ('Kajiado, Kenya','Kajiado, Kenya incl. Environmental Samples'):
        add_b_dis_widg.description = 'Sheep disease risk household level:'
    elif change.new == 'Kiambu, Kenya':
        add_b_dis_widg.description = 'Sheep disease risk household level:'        
input_data.observe(dis_pri_2_change, names='value')

add_g_dis_widg = widgets.FloatSlider(value=0,min=-2,max=2,step=1,description='Goat disease risk household level:',readout_format='+d',continuous_update=False)
# The below will change add_g_dis_widg widget description based on which region is selected
def dis_pri_3_change(change):
    if change.new == 'Gujarat, India':
        add_g_dis_widg.description = 'Goat disease risk household level:'
    elif change.new == 'Pune, India':
        add_g_dis_widg.description = 'Chicken disease risk household level:'
    elif change.new in ('Kajiado, Kenya','Kajiado, Kenya incl. Environmental Samples'):
        add_g_dis_widg.description = 'Goat disease risk household level:'
    elif change.new == 'Kiambu, Kenya':
        add_g_dis_widg.description = 'Chicken disease risk household level:'        
input_data.observe(dis_pri_3_change, names='value')

add_mix_anti_widg = widgets.FloatSlider(value=0,min=-1,max=1,step=0.05,description='Mixing antibiotics in the same animal:',readout_format='+.0%',continuous_update=False)
add_not_comp_widg = widgets.FloatSlider(value=0,min=-1,max=1,step=0.05,description='Not completing course:',readout_format='+.0%',continuous_update=False)
add_not_meas_widg = widgets.FloatSlider(value=0,min=-1,max=1,step=0.05,description='Not measuring dose accurately:',readout_format='+.0%',continuous_update=False)
add_use_prev_widg = widgets.FloatSlider(value=0,min=-1,max=1,step=0.05,description='Using for prevention:',readout_format='+.0%',continuous_update=False)

add_drink_mlk_widg = widgets.FloatSlider(value=0,min=-1,max=1,step=0.05,description='Drink milk from treated animals:',readout_format='+.0%',continuous_update=False)
add_eat_meat_widg = widgets.FloatSlider(value=0,min=-1,max=1,step=0.05,description='Eat meat from treated animals:',readout_format='+.0%',continuous_update=False)



# Layout map widgets

title_box = widgets.HBox([title])

in_box = widgets.HBox([input_data])

tran_box = widgets.HBox([add_dung_widg, add_clean_widg, add_visit_widg])

access_box = widgets.HBox([add_trained_widg,add_freq_widg])

herd_size_box = widgets.HBox([add_c_herd_widg,add_b_herd_widg,add_g_herd_widg])
herd_dis_box = widgets.HBox([add_c_dis_widg,add_b_dis_widg,add_g_dis_widg])

disease_box = widgets.VBox([herd_size_box,herd_dis_box])

use_anti_box = widgets.HBox([add_mix_anti_widg,add_not_comp_widg,add_not_meas_widg])
use_risk_box = widgets.HBox([add_use_prev_widg,add_drink_mlk_widg,add_eat_meat_widg])

use_box = widgets.VBox([use_anti_box,use_risk_box])

accordion = widgets.Accordion(children=[tran_box, access_box, disease_box, use_box], selected_index=None)
accordion.set_title(0, 'Transmission component data interaction (+/- percentage of "Yes" in communities by variable)')
accordion.set_title(1, 'Access component data interaction (+/- "Use of trained professional risk level" and "Antibiotic purchase frequency" per household)')
accordion.set_title(2, 'Disease component data interaction (+/- "Herd size" and "Disease risk level" by animal per household)')
accordion.set_title(3, 'Use component data interaction (+/- percentage of "Yes" in communities by variable)')
#ui = widgets.VBox([title_box, in_box,tran_box,access_box,herd_size_box,herd_dis_box,use_anti_box,use_risk_box])

map_ui = widgets.VBox([title_box, in_box,accordion])

map_out = widgets.interactive_output(AMR_tool, {'input_data': input_data,\
                                            'add_dung_widg': add_dung_widg,'add_clean_widg': add_clean_widg,'add_visit_widg': add_visit_widg,\
                                            'add_freq_widg': add_freq_widg, 'add_trained_widg': add_trained_widg,\
                                           'add_c_herd_widg':add_c_herd_widg,'add_b_herd_widg':add_b_herd_widg,'add_g_herd_widg':add_g_herd_widg,\
                                           'add_c_dis_widg':add_c_dis_widg,'add_b_dis_widg':add_b_dis_widg,'add_g_dis_widg':add_g_dis_widg,\
                                           'add_mix_anti_widg':add_mix_anti_widg,'add_not_comp_widg':add_not_comp_widg,'add_not_meas_widg':add_not_meas_widg,\
                                           'add_use_prev_widg':add_use_prev_widg,'add_drink_mlk_widg':add_drink_mlk_widg,'add_eat_meat_widg':add_eat_meat_widg})

# Build data table widgets

table_title = widgets.HTML(value="<p style='font-size:30px;text-align:center;'>Underlying data table views, without user-manipulation</p>")

w = widgets.Select(options=community_list)

### The following code links a change of map input data to the communities list 
def handle_slider_change(change):
    w.options = community_list
input_data.observe(handle_slider_change, names='value')
###

v = widgets.RadioButtons(options=look_item, description='Features')
u = widgets.RadioButtons(options=gran_lev, description='Granularity')

manip_table_title = widgets.HTML(value="<p style='font-size:30px;text-align:center;'>User-manipulated data table view</p>")
manip_table_out = widgets.interactive_output(look_manip, {'w': w,'input_data': input_data,\
                                            'add_dung_widg': add_dung_widg,'add_clean_widg': add_clean_widg,'add_visit_widg': add_visit_widg,\
                                            'add_freq_widg': add_freq_widg, 'add_trained_widg': add_trained_widg,\
                                           'add_c_herd_widg':add_c_herd_widg,'add_b_herd_widg':add_b_herd_widg,'add_g_herd_widg':add_g_herd_widg,\
                                           'add_c_dis_widg':add_c_dis_widg,'add_b_dis_widg':add_b_dis_widg,'add_g_dis_widg':add_g_dis_widg,\
                                           'add_mix_anti_widg':add_mix_anti_widg,'add_not_comp_widg':add_not_comp_widg,'add_not_meas_widg':add_not_meas_widg,\
                                           'add_use_prev_widg':add_use_prev_widg,'add_drink_mlk_widg':add_drink_mlk_widg,'add_eat_meat_widg':add_eat_meat_widg})


env_table_title = widgets.HTML(value="<p style='font-size:30px;text-align:center;'>Environmental Sample data table view</p>")
loc_list = widgets.Select(options=location_list)
env_table_ui = widgets.VBox([env_table_title,loc_list])

env_table_out = widgets.interactive_output(look_tab_samp, {'loc_list': loc_list,'input_data': input_data})

### The following code links a change of map input data to the communities list 
def handle_env_change(change):
    loc_list.options = location_list
input_data.observe(handle_env_change, names='value')
###




# Layout data table widgets

table_controls = widgets.HBox([w,v,u])
table_ui = widgets.VBox([table_title,table_controls])
table_out = widgets.interactive_output(look_tab, {'w': w, 'v': v, 'u': u,'input_data': input_data})


contact_deets = widgets.HTML(value="<p style='font-size:14px;text-align:right;'>Tool built by <a href='mailto:joshua.stephenson@live.co.uk'>Joshua Stephenson</a></p>")

display(map_ui, map_out, manip_table_title, manip_table_out, table_ui, table_out, env_table_ui, env_table_out, contact_deets)

Output()

HTML(value="<p style='font-size:30px;text-align:center;'>User-manipulated data table view</p>")

Output()

Output()

Output()

HTML(value="<p style='font-size:14px;text-align:right;'>Tool built by <a href='mailto:joshua.stephenson@live.c…